In [19]:
def wind_info(the_date):

    dfx = pd.DataFrame({'latitude':[40.6442],'longitude':[111.9522]},\
             index=  ['Salt Lake County'])

    start_date = datetime.strptime(the_date, '%Y-%m-%d')
    end_date = start_date + timedelta(days=10)

    #final_date = datetime.datetime.strptime("2024-03-20", '%Y-%m-%d')

    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": list(dfx.latitude.values),
        "longitude": list(dfx.longitude.values),
        "start_date": datetime.strftime(start_date, '%Y-%m-%d'),
        "end_date": datetime.strftime(start_date, '%Y-%m-%d'),
        "daily": ["wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant",],
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
        "timezone": "America/Denver"
    }
    responses = requests.get(url, params=params)
    return responses.json()['daily']

In [20]:
import datetime
import requests
import json
import datetime
#responses = wind_info(the_date='2023-06-10')

In [21]:
responses

NameError: name 'responses' is not defined

In [3]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta  # Import both datetime and timedelta
from joblib import load
import numpy as np
from sklearn.neighbors import NearestNeighbors
import requests
import json

def wind_info(the_date):
    dfx = pd.DataFrame({'latitude': [40.6442], 'longitude': [111.9522]}, index=['Salt Lake County'])
    
    start_date = datetime.strptime(the_date, '%Y-%m-%d')
    end_date = start_date + timedelta(days=10)

    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": dfx.iloc[0].latitude,  # Use scalar values for latitude and longitude
        "longitude": dfx.iloc[0].longitude,
        "start_date": start_date.strftime('%Y-%m-%d'),
        "end_date": end_date.strftime('%Y-%m-%d'),
        "daily": ["wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
        "timezone": "America/Denver"
    }
    
    response = requests.get(url, params=params)
    daily_data = response.json().get('daily', {})  # Ensure 'daily' key exists and has a default value
    return daily_data

def find_nearest_sensor_pm_value(latitude, longitude, df, pm_type='pm2'):
    coords = df[['latitude', 'longitude']].values
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(coords)
    distances, indices = nbrs.kneighbors([[latitude, longitude]])
    nearest_index = indices[0][0]
    nearest_pm_value = df.iloc[nearest_index][pm_type]
    return nearest_pm_value

def pm25_predict(input_lat, input_lng, input_date):
    url_base = '../static/data/'

    input_lng = round(input_lng, 5)
    input_lat = round(input_lat, 5)

    predict_input = pd.DataFrame({'longitude': [input_lng], 'latitude': [input_lat], 'date': [input_date]})

    begin_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    end_date_obj = begin_date_obj + timedelta(days=1)
    end_date = end_date_obj.strftime("%Y-%m-%d")

    connection = sqlite3.connect(f'{url_base}sensors_readings_2016_present.db')
    sql_query = """
    SELECT sensor_id, pm2, pm10, latitude, longitude
    FROM sensors_readings
    WHERE date(date) BETWEEN ? AND ?
    """
    df = pd.read_sql_query(sql_query, connection, params=(input_date, end_date))
    connection.close()

    df_cat = pd.read_csv(f'{url_base}sensor_categories.csv')
    df = df.merge(df_cat, on='sensor_id')
    nearest_pm2_5_value = find_nearest_sensor_pm_value(input_lat, input_lng, df, 'pm2')
    nearest_pm10_value = find_nearest_sensor_pm_value(input_lat, input_lng, df, 'pm10')

    url = f'{url_base}centroids_merge.csv'
    temp = pd.read_csv(url)
    temp['latitude'] = temp['latitude'].round(5)
    temp['longitude'] = temp['longitude'].round(5)

    predict_input = predict_input.merge(temp, on=['latitude', 'longitude'])
    predict_input['month'] = begin_date_obj.month
    # Ensure there's a 'county' column in predict_input for the following operations to work
    predict_input['avg_pm2'] = df.loc[df['county'] == predict_input['county'].iloc[0], 'pm2'].values[0]
    predict_input['avg_pm10'] = df.loc[df['county'] == predict_input['county'].iloc[0], 'pm10'].values[0]

    category_map = {'red': 1, 'orange': 2, 'green': 3, 'blue': 4}
    county_map = {'Salt Lake County': 1, 'Weber County': 2, 'Davis County': 3}
    predict_input['county_encoded'] = predict_input['county'].map(county_map)
    predict_input['category_encoded'] = predict_input['category'].map(category_map)

    df = predict_input[['latitude', 'longitude', 'avg_pm10', 'avg_pm2', 'county_encoded', 'category_encoded', 'month']]
    df['nearest_pm2_5'] = nearest_pm2_5_value
    df['nearest_pm10'] = nearest_pm10_value
    print(input_date)
    responses = wind_info(input_date)
    # Ensure 'responses' contains the expected keys before assignment
    df['wind_direction_10m_dominant'] = responses.get('wind_direction_10m_dominant', [np.nan])[0]
    df['wind_gusts_10m_max'] = responses.get('wind_gusts_10m_max', [np.nan])[0]
    df['wind_speed_10m_max'] = responses.get('wind_speed_10m_max', [np.nan])[0]

    preprocessor = load(f'{url_base}preprocessor.joblib')
    X_new_transformed = preprocessor.transform(df)

    return X_new_transformed


In [4]:
lat=40.74591050575336
lng=-111.89694078602857
the_date='2020-12-08'
df = pm25_predict(lat,lng,the_date)

/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_6204/2579130921.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_pm2_5'] = nearest_pm2_5_value
/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_6204/2579130921.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_pm10'] = nearest_pm10_value


2020-12-08


/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_6204/2579130921.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wind_direction_10m_dominant'] = responses.get('wind_direction_10m_dominant', [np.nan])[0]


In [5]:
df

array([[-0.39912935,  0.15767031, 30.94393607, 30.78767219, -0.13268305,
        -0.11411947, -0.23836798, -0.07337826,  0.47834662,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ]])

In [30]:

from sklearn.metrics import mean_squared_error, mean_absolute_error
print("Mean Absolute Error (MAE):", mean_absolute_error(df.nearest_pm2_5, df.pm2))
print("Mean Squared Error (MSE):", mean_squared_error(df.nearest_pm2_5, df.pm2))
print("Root Mean Squared Error (RMSE):", mean_squared_error(df.nearest_pm2_5, df.pm2, squared=False))
print("R-squared Score:", r2_score(df.nearest_pm2_5, df.pm2))

AttributeError: 'numpy.ndarray' object has no attribute 'nearest_pm2_5'

In [2]:
lat= 41.14339925293028
lng=  -112.029677791514445
the_date = '2023-04-01'

#41.118140	-112.031425	62.482979	57.820677	3	1	2

In [313]:
pm25_predict(lat,lng,the_date)

In [175]:
xdf = pd.read_csv('../static/data/centroids_data.csv')

In [29]:
pd.read_csv('../static/data/centroids_data.csv').to_json('../static/data/centroids_data.json', orient='records')


In [39]:
round(lng,5)

-111.89747

In [46]:
url = '../static/data/centroid_prep.csv'
temp_centroid = pd.read_csv(url)
temp_centroid['latitude'] = temp_centroid['latitude'].round(5)
temp_centroid['longitude'] = temp_centroid['longitude'].round(5)

In [45]:
temp_centroid

,Unnamed: 0,OBJECTID,GEOID,Tract,Low,Lowmod,Lowmod_pct,geometry,longitude,latitude,County_encoded,category_encoded,category_blue,category_green,category_orange,category_red,county_Davis County,county_Salt Lake County,county_Weber County
0,0,199060,490111251021,125102,120,340,0.1833,POINT (-111.88080951152304 41.08032456289652),-111.880810,41.080325,0,0,True,False,False,False,True,False,False
1,1,199061,490111251022,125102,245,490,0.1892,POINT (-111.88003663259194 41.04248386241919),-111.880037,41.042484,0,0,True,False,False,False,True,False,False
2,2,199062,490111251031,125103,370,850,0.3131,POINT (-111.9563088264819 41.13919426676789),-111.956309,41.139194,0,1,False,True,False,False,True,False,False
3,3,199063,490111251032,125103,430,1000,0.4264,POINT (-111.94339340147886 41.11547671683034),-111.943393,41.115477,0,1,False,True,False,False,True,False,False
4,4,199064,490111251041,125104,165,475,0.1967,POINT (-111.91575298025643 41.12877405887183),-111.915753,41.128774,0,0,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,907,200607,490572112012,211201,70,170,0.1560,POINT (-111.92049976396636 41.15200853107149),-111.920500,41.152009,2,0,True,False,False,False,False,False,True
908,908,200608,490572112013,211201,365,975,0.4194,POINT (-111.93262761353571 41.156133092177804),-111.932628,41.156133,2,1,False,True,False,False,False,False,True
909,909,200609,490572112021,211202,115,270,0.3354,POINT (-111.96410356104677 41.16503488421328),-111.964104,41.165035,2,1,False,True,False,False,False,False,True
910,910,200610,490572112022,211202,620,1710,0.3963,POINT (-111.95158835880763 41.15407118264731),-111.951588,41.154071,2,1,False,True,False,False,False,False,True
